# Neural Network with 2D Circle Data
- Neural Network with 5 layers
- 2D data f(x,y) -> R

## 1.Import Required Libraries

In [1]:
!pip3 install visdom

     |████████████████████████████████| 676 kB 10.5 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=7b1f62b0b48b9b53346182e222ee30a082968643a8001d467dc57746ce47ae89
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=058b018b267ed413ec0fb013a3b12702f11d564baea42271e4151209cab63f2a
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom

In [3]:
# 코렙에서 visdom 사용 방법

# !npm install -g npm to update!
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

import time
time.sleep(5)
! cat url.txt

import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt
# 아래 출력 메세지의 Your url is 부분의 url 클릭함
# 나타난 웹 페이지의 'click to countinue' 버튼 클릭함

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.236s
your url is: https://giant-cat-41.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.97ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.46ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.78ms


## 2. Data Generation

In [4]:
num_data = 1000
num_epoch = 400

x = init.uniform(torch.Tensor(num_data, 1), -10, 10)
y = init.uniform(torch.Tensor(num_data, 1), -10, 10)
z = x**2 + y**2

x_noise = init.normal(torch.FloatTensor(num_data, 1), std=0.5)
y_noise = init.normal(torch.FloatTensor(num_data, 1), std=0.5)
z_noise = x_noise**2 + y_noise**2

data_noise = torch.cat([x_noise, y_noise, z_noise], 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  if __name__ == '__main__':


In [5]:
# visualize data with visdom

win_1=vis.scatter(
        X=data_noise,
        opts=dict(
            markersize=5,
            markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=[51,153,255]*np.ones(shape=[num_data,3]))
            )
        )


## 3. Model & Optimizer

In [6]:
# fully connected model with 5 hidden layer

model = nn.Sequential(
        nn.Linear(2,20),
        nn.ReLU(),
        nn.Linear(20,10),
        nn.ReLU(),
        nn.Linear(10,5),
        nn.ReLU(),
        nn.Linear(5,5),
        nn.ReLU(),
        nn.Linear(5,1),
    ).cuda()

loss_func = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

## 4. Train 

In [7]:
input_data = torch.cat([x,y],1).cuda()
label = z_noise.cuda()

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(input_data))
    loss = loss_func(output,Variable(label))
    loss.backward()
    optimizer.step()
    print(loss)
    
    data = torch.cat([input_data.cpu(),output.cpu().data],1)
    
    if i % (num_epoch / 10) == 0:
        win_2 =vis.scatter(
                X=data,
                opts=dict(
                markersize=5,
                markercolor=np.ndarray(shape=[num_data,3],
                                       dtype=float,
                                       buffer=128*np.ones(shape=[num_data,3]))
            )
    )

tensor(0.3972, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3951, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3931, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3912, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3895, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3878, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3863, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3849, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3835, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3824, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3813, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3806, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3800, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3795, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3790, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3784, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3779, device='cuda:0', grad_fn=<L1LossBackward0

In [8]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[ 0.0521, -0.3666],
        [-0.7147,  0.1756],
        [-0.6593,  0.6156],
        [ 0.4198,  0.3341],
        [ 0.4313, -0.6526],
        [-0.4929,  0.1535],
        [ 0.2073, -0.4682],
        [ 0.1417, -0.6339],
        [-0.2760, -0.6037],
        [ 0.4803,  0.0368],
        [-0.2939,  0.6368],
        [-0.4816,  0.1790],
        [ 0.5495, -0.5081],
        [ 0.2744,  0.2412],
        [-0.2803,  0.7124],
        [ 0.3708, -0.8204],
        [-0.1324, -0.2580],
        [ 0.6633, -0.6474],
        [ 0.2127, -0.0672],
        [-0.1142, -0.6114]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.7303,  0.6446,  0.4797,  0.1298,  0.0393,  0.6707,  0.1793, -0.6163,
        -0.2085,  0.4995, -0.0629, -0.3714, -0.3381, -0.3325,  0.5516, -0.6679,
        -0.1040,  0.0684,  0.0412,  0.1362], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[-8.4004e-02,  6.1461e-02, -1.7537e-01,  2.1441e-01, -1.3390e-01,
          

## 5. Visualize Trained Output

In [ ]:
data = torch.cat([input_data.cpu(),output.cpu().data],1)

win_2 = vis.scatter(
    X = data,
    opts=dict(        
        markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=128*np.ones(shape=[num_data,3])),
        markersize=5,
    ),
)


## 6. Visualize Loss Graph

In [ ]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win3 = vis.line(
    X = x.reshape(x.shape[0]),
    Y = loss_data.reshape(loss_data.shape[0]),
)